In [3]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np 
from multiprocessing import Pool
import time 

import sys
from pathlib import Path

try:
    sys.path.append(str(Path().cwd().parent.parent))
except IndexError:
    pass

#import utils.audio as a
#import utils.plots as plots
import utils as utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
LABELS = "../../datasets/dataset_laughter_single_22050.csv"
input_df = utils.misc.readCSV(LABELS)
input_df.head()


,Filename,Gender,Count,Type,Enthusiasm
0,_547JTmvb9g_enc,female,1,laughter,3
1,1-1791-A-26_enc,mix,2,giggle,1
2,1-33658-A-26_enc,female,1,laughter,1
3,1-36164-A-26_enc,female,1,giggle,1
4,1-36164-B-26_enc,female,1,laughter,2


### Females

In [5]:
# get female laughter
idx_female = input_df.index[input_df['Gender'] == 'female']
females = input_df.iloc[idx_female]
print(females)
print(females.iloc[0])
print(input_df.iloc[0].Filename)

             Filename  Gender  Count      Type  Enthusiasm
0     _547JTmvb9g_enc  female      1  laughter           3
2    1-33658-A-26_enc  female      1  laughter           1
3    1-36164-A-26_enc  female      1    giggle           1
4    1-36164-B-26_enc  female      1  laughter           2
5    1-72695-A-26_enc  female      1  laughter           2
6    1-73123-A-26_enc  female      1  laughter           1
8    2-79775-A-26_enc  female      1  laughter           2
9   2-109759-A-26_enc  female      1  laughter           1
10  2-109759-B-26_enc  female      1  laughter           2
11  3-107123-A-26_enc  female      1  laughter           3
12  3-110536-A-26_enc  female      1  laughter           2
13  3-118487-A-26_enc  female      1  laughter           3
15  3-152912-A-26_enc  female      1  laughter           2
16  3-152997-A-26_enc  female      1  laughter           1
17  4-181599-A-26_enc  female      1  laughter           3
24    D-1RsAdzIcc_enc  female      1    giggle          

### Males

In [6]:
# get male laughter
idx_male = input_df.index[input_df['Gender'] == 'male']
males = input_df.iloc[idx_male]
print(males)
print(males.iloc[0])
print(input_df.iloc[0].Filename)

             Filename Gender  Count      Type  Enthusiasm
7    2-79769-A-26_enc   male      1  laughter           2
14  3-119459-A-26_enc   male      1  laughter           3
20  5-244526-A-26_enc   male      1  laughter           3
21  5-259514-A-26_enc   male      1  laughter           2
22  5-263775-A-26_enc   male      1    giggle           2
23  5-263775-B-26_enc   male      1  laughter           3
25    THhP1idrWXA_enc   male      3  laughter           3
Filename      2-79769-A-26_enc
Gender                    male
Count                        1
Type                  laughter
Enthusiasm                   2
Name: 7, dtype: object
_547JTmvb9g_enc


In [7]:
PATH = '../../datasets/dataset_laughter_single_22050/'
SR = 22050 

dataset = []
for i in np.arange(len(input_df)):
    audio, sr = utils.audio.loadAudio(PATH + input_df.iloc[i].Filename + '.wav', sr=SR, fix_length=True, length=10)
    dataset.append(audio)

### Create N Female audio files without alpha, beta 

In [8]:
N = 500

t = time.time()
if not os.path.exists('female_audios'):
    os.makedirs('female_audios')

PATH = "female_audios/"
len_females = len(females)

# in percent
min_speed, max_speed = 0.9, 1.2
speed_range = (max_speed - min_speed)  # 2
max_noise = 0.001 
min_pitch, max_pitch = -4, 4
audio = np.zeros_like(dataset[0])

idx_list = utils.misc.getFemaleList(input_df, N)

# add radial decay
alpha=np.random.rand(N)
beta=np.reciprocal(np.sqrt(np.arange(1, N+1)))

#pdb.set_trace()
noise_factor = list(np.random.rand(N) * max_noise)
shift_max = np.random.rand(N) * 3 # in seconds
shift = [np.random.randint(0, int(SR * i)) for i in shift_max]
direction = ["left" if np.random.randint(0, 2) else "right" for i in np.arange(N)] 
pitch_factor = list(np.random.randint(min_pitch, max_pitch, N)) # pitch > 0 higher pitch, else deeper voice
speed_factor = list(np.random.rand(N) * speed_range + min_speed)
data = [dataset[idx] for idx in idx_list] 

#create arguments as list for parallel computing
func_args = [(data[i], SR, noise_factor[i], shift[i], direction[i], pitch_factor[i], speed_factor[i]) for i in np.arange(N)]

#multiprocessing
with Pool(8) as p:
    out = p.starmap(utils.audio.augmentAudio, func_args)

#pdb.set_trace()
np.save(PATH+"female_audios_500.npy", np.array(out)) # two channel audio
elapsed = time.time() - t
print(elapsed)

36.594746828079224


### Create N Male audio files without alpha, beta 


In [9]:
N = 500

t = time.time()
if not os.path.exists('male_audios'):
    os.makedirs('male_audios')

PATH = "male_audios/"
len_males = len(males)

# in percent
min_speed, max_speed = 0.9, 1.2
speed_range = (max_speed - min_speed)  # 2
max_noise = 0.001 
min_pitch, max_pitch = -4, 4
audio = np.zeros_like(dataset[0])

idx_list = utils.misc.getMaleList(input_df, N)

# add radial decay
alpha=np.random.rand(N)
beta=np.reciprocal(np.sqrt(np.arange(1, N+1)))

#pdb.set_trace()
noise_factor = list(np.random.rand(N) * max_noise)
shift_max = np.random.rand(N) * 3 # in seconds
shift = [np.random.randint(0, int(SR * i)) for i in shift_max]
direction = ["left" if np.random.randint(0, 2) else "right" for i in np.arange(N)] 
pitch_factor = list(np.random.randint(min_pitch, max_pitch, N)) # pitch > 0 higher pitch, else deeper voice
speed_factor = list(np.random.rand(N) * speed_range + min_speed)
data = [dataset[idx] for idx in idx_list] 

#create arguments as list for parallel computing
func_args = [(data[i], SR, noise_factor[i], shift[i], direction[i], pitch_factor[i], speed_factor[i]) for i in np.arange(N)]

#multiprocessing
with Pool(8) as p:
    out = p.starmap(utils.audio.augmentAudio, func_args)

#pdb.set_trace()
np.save(PATH+"male_audios_500.npy", np.array(out)) # two channel audio
elapsed = time.time() - t
print(elapsed)

36.081870555877686
